In [ ]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
import numpy as np

# Define the Excel file path
file_path = r"file_path"

# Step 1: Extract relevant data from the Excel sheet
# Load the "Chaku Foods - Data Collection" sheet
data = pd.read_excel(file_path, sheet_name='Chaku Foods - Data Collection')

# Select relevant columns for the `farmer` table
farmer_data = data[[
    'date', 'last_name', 'first_name', 'nationality', 'sex', 'dob', 'id_type', 'id_number',
    'id_doc_number', 'id_image_url', 'farmer_image', 'marital_status', 'contact_number',
    'mobile_money_number', 'association', 'location', 'global_gap_certified', 'usda_organic_certified'
]].copy()

# Rename columns to match database schema
farmer_data.rename(columns={
    'date': 'start_date',
    'farmer_image': 'farmer_image_url'
}, inplace=True)

# Replace NaN values with None for proper handling in MySQL
farmer_data = farmer_data.map(lambda x: None if pd.isna(x) else x)

# Validate ENUM values
valid_marital_status = ['Single', 'Married', 'Divorced', 'Widowed']
valid_certifications = ['Yes', 'No, but interested and qualified', 'No and not interested']

farmer_data['marital_status'] = farmer_data['marital_status'].apply(lambda x: x if x in valid_marital_status else None)
farmer_data['global_gap_certified'] = farmer_data['global_gap_certified'].apply(lambda x: x if x in valid_certifications else None)
farmer_data['usda_organic_certified'] = farmer_data['usda_organic_certified'].apply(lambda x: x if x in valid_certifications else None)

# Align data types with schema
farmer_data['start_date'] = pd.to_datetime(farmer_data['start_date'], errors='coerce')
farmer_data['dob'] = farmer_data['dob'].apply(lambda x: pd.to_datetime(x, errors='coerce') if pd.notna(x) else None)
farmer_data['is_mobile_money_same'] = farmer_data['contact_number'] == farmer_data['mobile_money_number']

# Ensure strings do not exceed VARCHAR limits and strip whitespace
varchar_columns = {
    'last_name': 100,
    'first_name': 100,
    'nationality': 50,
    'id_type': 50,
    'id_number': 100,
    'id_doc_number': 50,
    'id_image_url': 255,
    'farmer_image_url': 255,
    'contact_number': 20,
    'mobile_money_number': 20,
    'association': 100,
    'location': 100
}
for col, max_len in varchar_columns.items():
    farmer_data[col] = farmer_data[col].apply(lambda x: str(x).strip()[:max_len] if x else None)

# Ensure CHAR(1) fields are limited to one character
farmer_data['sex'] = farmer_data['sex'].apply(lambda x: str(x).strip()[:1] if x else None)

# Remove rows with 'Unknown' in both first_name and last_name
farmer_data = farmer_data[~((farmer_data['first_name'] == 'Unknown') & (farmer_data['last_name'] == 'Unknown'))]

# Reset AUTO_INCREMENT for the farmer table
def reset_auto_increment(connection, table_name):
    cursor = connection.cursor()
    cursor.execute(f"ALTER TABLE {table_name} AUTO_INCREMENT = 1;")
    connection.commit()
    print(f"AUTO_INCREMENT for {table_name} has been reset.")

# Step 2: Connect to the MySQL database
def create_connection():
    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',  # Replace with your MySQL username
            password='*****',  # Replace with your MySQL password
            database='*****'  # Replace with your database name
        )
        if connection.is_connected():
            print("Connected to the database")
            return connection
    except Error as e:
        print(f"Error: {e}")
        return None

# Step 3: Insert data into the `farmer` table
def insert_farmer_data(connection, farmer_data):
    cursor = connection.cursor()
    insert_query = """
    INSERT INTO farmer (
        start_date, last_name, first_name, nationality, sex, dob, id_type, id_number,
        id_doc_number, id_image_url, farmer_image_url, marital_status, contact_number,
        mobile_money_number, is_mobile_money_same, association, location,
        global_gap_certified, usda_organic_certified
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    for _, row in farmer_data.iterrows():
        cursor.execute(insert_query, (
            row['start_date'], row['last_name'], row['first_name'], row['nationality'],
            row['sex'], row['dob'], row['id_type'], row['id_number'], row['id_doc_number'],
            row['id_image_url'], row['farmer_image_url'], row['marital_status'],
            row['contact_number'], row['mobile_money_number'], row['is_mobile_money_same'],
            row['association'], row['location'], row['global_gap_certified'],
            row['usda_organic_certified']
        ))
    connection.commit()
    print("Data inserted into the `farmer` table successfully.")

# Main execution block
if __name__ == "__main__":
    db_connection = create_connection()
    if db_connection:
        try:
            # Reset AUTO_INCREMENT for the farmer table
            reset_auto_increment(db_connection, 'farmer')

            # Insert data into the `farmer` table
            insert_farmer_data(db_connection, farmer_data)
        except Error as e:
            print(f"Error during data insertion: {e}")
        finally:
            db_connection.close()
            print("Database connection closed.")
